In [55]:
import numpy as np
import plotly.graph_objects as go
from ipywidgets import interact

import os, sys
sys.path.append('../..')

import lti_disc, kf_predict, kf_update, rts_smooth

In [56]:
A = np.array([[1,1],[0,1]])
H = np.array([1, 0])

sd_r = 10
T = 100
R = sd_r**2

X = np.zeros((T,2))
y = np.zeros((T,1))

mean1 = np.zeros(2)
cov1 = np.eye(2)
Q = np.array([[1/100, 0],[0, 1]])
X[0] = np.random.multivariate_normal(mean1,cov1,1)
y[0] = H @ X[0]
# print(y[0])
# y_teem = H @ X[0].T
# print(y_teem)
for ii in range(T-1):
    X[ii + 1] = A @ X[ii] + np.random.multivariate_normal(mean1, Q, 1)
    y[ii + 1] = H @ X[ii] + sd_r * np.random.normal()

plot $x_k$ and $\hat{x_k}$ and $y_k$

In [57]:
TT = np.array(range(T))
fig1 = go.FigureWidget()
fig1.add_scatter(x=TT, y=X[:,0], name='Real signal', opacity=0.7)
fig1.add_scatter(x=TT, y=X[:,1], name='Derivative', opacity=0.7)
fig1.add_scatter(x=TT, y=y[:,0],  mode='markers', marker_symbol='circle-open',  name='Measurements')

FigureWidget({
    'data': [{'name': 'Real signal',
              'opacity': 0.7,
              'type': 'scatt…

Use the Kalman filter for computing the state estimates $\boldsymbol{m}_k$ 

In [76]:
M = np.zeros((2,1))
P = np.eye(2)

# Track and animate
MM = np.zeros((len(y), len(M), 1))
PP = np.zeros((len(y), len(M), len(M)))

for k in range(T):
    M, P = kf_predict.kf_predict(M, P, A, Q)
    M, P, *_ = kf_update.kf_update(M, P, y[k], H, R)

    MM[k], PP[k] = M, P

In [77]:
fig2 = go.FigureWidget()
fig2.add_scatter(x=TT, y=X[:,0], name='Real signal', opacity=0.7)
fig2.add_scatter(x=TT, y=X[:,1], name='Derivative', opacity=0.7)
fig2.add_scatter(x=TT, y=y[:,0],  mode='markers', marker_symbol='circle-open',  name='Measurements')
fig2.add_scatter(x=TT, y=MM[:,0,0], line_dash='dash', name='Filtered estimate')
fig2.add_scatter(x=TT, mode='lines', name='Filtered estimate')
fig2.layout.update(title='Estimating a noisy sine signal with Kalman filter', height=600)

@interact(step=(1, len(TT), 1))
def update(step=2):
    with fig2.batch_update():
        fig2.data[-1].y=MM[:int(step),0,0]
fig2

interactive(children=(IntSlider(value=2, description='step', min=1), Output()), _dom_classes=('widget-interact…

FigureWidget({
    'data': [{'name': 'Real signal',
              'opacity': 0.7,
              'type': 'scatt…

In [79]:
# Errors
print('RMS errors:')
print('KF = {0:4f} \n'.format(*np.sqrt(np.mean(np.array([MM[:,0,0]-X[:,0]])**2, axis=1))))

RMS errors:
KF = 9.179424 

